In [1]:
import numpy as np　#数値計算統計解析やデータ分析などで、 データ加工や演算をスムーズに行うためのライブラリ
import matplotlib.pyplot as plt　#暗黙的かつ自動的に図形や軸を作成するインターフェース
import pandas as pd #データの読込や並べ替え、欠損値（欠けている項目値）の補完などを行うことができるライブラリ
import random #ランダムな要素や乱数を使う
%matplotlib inline
import seaborn as sns #データの可視化(グラフ作成)を行うPythonライブラリです。 Pythonで有名な可視化ライブラリであるmatplotlibのラッパー関数(内包プログラム)

import requests #HTTP通信ライブラリ
import json　#jsonファイルからの読み込み、データの文字列 ⇔ 辞書型変換、ファイルへの保存を行う
import re #正規表現を行うモジュール





a


In [ ]:
# APIの仕様は
url_path = "https://www.land.mlit.go.jp/webland/api/TradeListSearch?from=20171&to=20175&area=13"
request_result = requests.get(url_path)
data_json = request_result.json()["data"]and
data_pd

In [ ]:
data_from_csv = pd.read_csv("13_Tokyo_20171_20184.csv", encoding='cp932')
data_from_csv.head(5)
data_from_csv["A"].unique()
data_used_apartment = data_from_csv.query('種類 == "中古マンション"')
data_used_apartment.shape
data_used_apartment.isnull().sum()
data_added_dummies["面積（㎡）"] = data_added_dummies["面積（㎡）"].astype(float)
plt.hist(data_added_dummies["取引価格（総額）"])
plt.show()

from sklearn.linear_model import LinearRegression # 線形回帰をする用のモジュールを呼び出し
from sklearn.model_selection import train_test_split # 検証用に学習データとテストデータを分割してくれるモジュールを呼び出し

print(data_from_csv.iloc[0])



In [ ]:
!curl -L http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz -O && tar xzvf ta-lib-0.4.0-src.tar.gz
!cd ta-lib && ./configure --prefix=/usr && make && make install && cd - && pip install ta-lib
!pip install tqdm
!pip install plotly --upgrade

def enable_plotly_in_cell():
    import IPython
    from plotly.offline import init_notebook_mode
    display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
    init_notebook_mode( connected=False )


    